# Regression: synthetic dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor  # for regression
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
import sklearn
import copy

import shap

from pred_diff import preddiff
from pred_diff.imputers import impute
import pred_diff.tools.plot as c_plt
from pred_diff.datasets.datasets_regression import SyntheticDataset
from pred_diff.tools import init_plt

Select plotting style

In [ ]:
# paper style
init_plt.update_rcParams(fig_width_pt=234.88*2)

# default
# plt.style.use('default')


### Set-up experiment
1. Specify mean and covariance for dataset


In [ ]:
# create synthetic dataset
cov = [[1, 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]
mean = np.array([0, 0, 0, 0])

2. Create custom target functions

In [ ]:
def f(x: np.ndarray) -> np.ndarray:
    x0 = x[:, 0]; x1 = x[:, 1]; x2 = x[:, 2]; x3 = x[:, 3]

    y_additive = x0 ** 2 + 3 * x1 + np.sin(np.pi * x2) - 0.5 * x3 ** 3
    y_interaction = 2 * np.sign(x0) * np.abs(x1)

    return y_additive + y_interaction

dataset = SyntheticDataset(function=f, mean=mean, cov=cov)

3. Select model

In [ ]:
model_selection = 'RandomForest'
# model_selection = 'FullyConnectedNetwork'
# model_selection = 'GaussianProcess'

4. Train model

In [ ]:
x_df, y_df = dataset.load_pd()
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x_df, y_df, test_size=0.1)
if model_selection == 'RandomForest':
    reg = RandomForestRegressor(n_estimators=100)
elif model_selection == 'FullyConnectedNetwork':
    reg = MLPRegressor([int(x_df.shape[0])])
elif model_selection == 'GaussianProcess':
    reg = GaussianProcessRegressor(1.0 * RBF() + WhiteKernel())
else:
    assert False, f'please enter a valid model_selection = {model_selection}'

reg.fit(x_train, y_train)
print(f"train data:     "
      f"score = {reg.score(x_train, y_train):.3f}\n"
      f"test data:      "
      f"score = {reg.score(x_test, y_test):.3f}")

## *PredDiff*
### Relevances and interactions
Figure 1: *PredDiff* contributions

In [ ]:
n_imputations = 200

imputer = impute.GaussianProcessImputer
explainer = preddiff.PredDiff(model=reg, df_train=copy.deepcopy(x_train), imputer_cls=imputer, 
                              fast_evaluation=True, n_group=50)

c_plt.scatter_m_plots(reg, df_train=copy.deepcopy(x_train), df_test=copy.deepcopy(x_test), imputer=imputer,
                      n_imputations=n_imputations)


# use interaction matrix
relevance_matrix = explainer.interaction_matrix(df_test=x_test, n_imputations=n_imputations)
c = pd.DataFrame(np.diagonal(relevance_matrix, axis1=1, axis2=2), columns=x_test.columns)
c_plt._scatter(c=c, x_df=x_test, method=f'Residual PredDiff, n={n_imputations}')

c_plt.scatter_2d_heatmap(x=x_test['0'], y=x_test['1'], relevance=relevance_matrix[:, 1, 0],
                         title='PredDiff - 0 vs. 1', axis_symmetric=True)

Figure 2: Computational dependence fo *PredDiff*

In [ ]:
for n_imputations in [5, 50, 200]:
    c_plt.plot_n_dependence(reg=reg, x_train=x_train, x_test=x_test, n_imputations=n_imputations)


### Comparison to SHAP
Appendix

In [ ]:
explainer = shap.TreeExplainer(reg)
shap_interaction_values = explainer.shap_interaction_values(x_test)
c_plt.scatter_2d_heatmap(x=x_test['0'], y=x_test['1'], relevance=shap_interaction_values[:, 1, 0],
                         title='SHAP TreeExplainer - 0 vs. 1', axis_symmetric=True)
residual_feat_contribution = np.diagonal(shap_interaction_values, axis1=1, axis2=2)

c = pd.DataFrame(residual_feat_contribution, columns=x_test.columns)
c_plt._scatter(c=c, x_df=x_test, method='SHAP - residual feature contributions')

c_plt.scatter_contributions(reg, x_test, method='TreeExplainer')
c_plt.scatter_contributions(reg, x_test, method='KernelExplainer', x_train=x_train)